# 2018-05-17 - Convolutional + BiRNN 

# CNN + BiRNN for MC
![Could not display RNN picture](http://slideplayer.com/slide/10066142/32/images/8/Recurrent+Neural+Networks.jpg)
Les réseaux de neurones bidirectionnels ont l'avantage sur les RNN qu'ils peuvent représenter des informations dans les deux sens, forward et backward, ce qui dans noter cas permet de mimer les connexions latérales de V1. Le traitement unitaire des colonnes de V1 est représenté par un CNN.

In [10]:
"""
Created on Tue May 15 11:25:41 2018

@author: hugo
"""
from torch.autograd import Variable
import torch 
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
import torch.nn.functional as F

#Transform
data_transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.Resize((64,64)),
    transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5,0.5), (0.5,0.5,0.5))])

#Train
train_set = datasets.ImageFolder(root='16_clouds_easy',
                                transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                             batch_size=8, shuffle=True,
                                             num_workers=1)

#Test
test_set = datasets.ImageFolder(root='16_clouds_easy_test',
                                transform=data_transform)
test_loader = torch.utils.data.DataLoader(test_set,
                                             batch_size=8,shuffle=False,
                                             num_workers=1)

###################################################################################

# Hyper-parameters
sequence_length = 22
input_size = 22
hidden_size = 128
num_layers = 2
num_classes = 16
batch_size = 8
num_epochs = 100
learning_rate = 0.003

###################################################################################

class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.conv = nn.Conv2d(1,6,20)
        self.pool = nn.MaxPool2d(2,2)

        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection
    
    def forward(self, x):
        print('Input %s' % str(x.size()))

        x = self.pool(F.relu(self.conv(x)))
        print('Convoluted %s' % str(x.size()))

        # Init
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)
        h0 = Variable(h0)
        c0 = Variable(c0)
        
        x = x[:,-1,:,:]
        print('Resized for RNN %s' % str(x.size()))
        
        # LSTM forward
        out, _ = self.rnn(x, h0)  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        print('After RNN %s' % str(out.size()))
        
        # LSTM output
        out = self.fc(out[:, -1, :])
        print('Reshaped for output %s \n'%  str(out.size()))

        return out

model = BiRNN(input_size, hidden_size, num_layers, num_classes)
print(model)

###################################################################################

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

###################################################################################

#plotting list
loss_list = []
time_list = []
t = 0

print("Start training")
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)

        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.data[0]))
            loss_list.append(loss.data[0])
            time_list.append(t)
            t+=1
            

BiRNN(
  (rnn): LSTM(22, 128, num_layers=2, batch_first=True, bidirectional=True)
  (conv): Conv2d(1, 6, kernel_size=(20, 20), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (fc): Linear(in_features=256, out_features=16, bias=True)
)
Start training
Input torch.Size([8, 1, 64, 64])
Convoluted torch.Size([8, 6, 22, 22])
Resized for RNN torch.Size([8, 22, 22])
After RNN torch.Size([8, 22, 256])
Reshaped for output torch.Size([8, 16]) 

Input torch.Size([8, 1, 64, 64])
Convoluted torch.Size([8, 6, 22, 22])
Resized for RNN torch.Size([8, 22, 22])
After RNN torch.Size([8, 22, 256])
Reshaped for output torch.Size([8, 16]) 

Input torch.Size([8, 1, 64, 64])
Convoluted torch.Size([8, 6, 22, 22])
Resized for RNN torch.Size([8, 22, 22])
After RNN torch.Size([8, 22, 256])
Reshaped for output torch.Size([8, 16]) 

Input torch.Size([8, 1, 64, 64])
Convoluted torch.Size([8, 6, 22, 22])
Resized for RNN torch.Size([8, 22, 22])


Process Process-8:
Traceback (most recent call last):
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hugo/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
KeyboardInterrupt
Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7fd7ecc8add8>>
Traceback (most recent call last):
  File "/home/hugo/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 333, in __del__
    self._shutdown_workers()
  File "/home/hugo/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 319, in _shutdown_workers
    self.data_queue.get()
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/queues.py", line 337, in get

KeyboardInterrupt: 

  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)


In [ ]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, sequence_length, input_size))
    labels = Variable(labels)

    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += predicted.eq(labels.data).sum()

print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(time_list, loss_list)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss for MNIST-RNN')
plt.show()